In [ ]:
from dolfin import *
from petsc4py import *
%matplotlib inline
import numpy as np

mesh = UnitSquareMesh(10,10)
V = FunctionSpace(mesh, "Lagrange", 2)
f = Constant(4.)
#g = FacetArea(mesh)

u = TrialFunction(V)
v = TestFunction(V)

a = inner(grad(u), grad(v))*dx
L = v*f*dx

def bdry(x):
    return near(x[0],0) or near(x[0],1)# or near(x[1],0) or near(x[1],1)

bc = DirichletBC(V, Constant(0.0), bdry)
u = Function(V)
A = assemble(a)
b = assemble(L)
bc.apply(A, b)
uv = u.vector()
_ = solve(A, uv, b)

#_ = plot(u)

In [ ]:
M = as_backend_type(A)
type(M)

In [ ]:
m = PETSc.Mat().create() #AIJ(10, nnz=True)
m.setType('aij')
m.setOption(19, 0)  #MAT_NEW_NONZERO_ALLOCATION_ERR, PETSC_FALSE

In [ ]:
PETSc.MAT_NEW_NONZERO_ALLOCATION_ERR = 19
PETSc.PETSc_FALSE = 0

In [ ]:
row_indices = np.array([1,2], dtype=np.intc)
col_indices = np.array([1,2,6], dtype=np.intc)
block = np.ones((len(row_indices), len(col_indices)), order='C', dtype=np.float)

M.mat().setOption(PETSc.MAT_NEW_NONZERO_ALLOCATION_ERR, PETSc.PETSc_FALSE)
M.set(block, row_indices, col_indices)
M.apply("insert")

In [ ]:
M.array().round(1)

In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
import nbimporter
from boundary import apply_dirichlet_hermite, apply_neumann_hermite

%matplotlib inline

In [ ]:
mesh = UnitSquareMesh(2, 2)
V = FunctionSpace(mesh, "Hermite", 3)

u = TrialFunction(V)
v = TestFunction(V)
n = FacetNormal(V.mesh()) ## unused


a = inner(grad(u), grad(v))*dx
f = project(Constant(-4.0), V)
g = project(Constant(1.0), V)
d = project(Constant(0.), V)

def top(x):
    return near(x[1], 1.)

class NeumannBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary #and not top(x)

exterior_facets = FacetFunction('uint', V.mesh(), value=0)
neu = NeumannBoundary()
neu.mark(exterior_facets, 1)

bc = DirichletBC(V, d, top)
L = f*v*dx(domain=V.mesh())
#bp = assemble(L)
Lneumann = L + g*v*ds(subdomain_data=exterior_facets, subdomain_id=1)

A = assemble(a)
b = assemble(Lneumann)

apply_dirichlet_hermite(A, b, bc)

u = Function(V)
uv = u.vector()
solve(A, uv, b)

nn = assemble(inner(n, grad(u))*ds(subdomain_data=exterior_facets, subdomain_id=1))
print("Error in boundary integral of normal derivative = |4 - nn| = %f" % np.abs(4 - nn))
_ = plot(u)

# More...

In [ ]:
import nbimporter
from boundary import plot_dofs, get_facets_vertices, find_hermite_boundary_dofs

In [ ]:
neumann_dofs = vertex_to_dof_map(V)[get_facets_vertices(exterior_facets, 1)]
plot_dofs(V, neumann_dofs)

In [ ]:
(bp.array() - b.array())[neumann_dofs].round(2)

In [ ]:
be = assemble(v*ds)
be.array().round(2)

In [ ]:
import matplotlib.pyplot as pl
import numpy as np

xx = np.linspace(0,1,100)
for y in np.linspace(0,1,5):
    pl.plot(xx, [u(x, y) for x in xx], label="%.1f" % y)
_ = pl.legend()

In [ ]:
pl.figure(figsize=(10,4))
pl.subplot(1,2,1)
plot(u.dx(0), title="dx", cmap='bone')
pl.subplot(1,2,2)
_ = plot(u.dx(1), title="dy", cmap='bone')

In [ ]:
import nbimporter

In [ ]:
from boundary import *

In [ ]:
d = {'a':1, 'b':2, 'c':3, '_d':4}
names = ['a', 'b', 'c']
all = ['a', 'b']
for name in set(names) - set(all):
    del d[name]
d